In [ ]:
# import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

!nvidia-smi
import platform
platform.python_version()

In [ ]:
!free

In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ''
# before execute any computation / allocation
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

import numpy as np 
import pandas as pd 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split, DataLoader

import matplotlib.pyplot as plt

### Training a BERT model

In [ ]:
it = 99

# from shutil import copyfile

import os, wandb

os.environ["WANDB_NOTEBOOK_NAME"] = "xxx"
project_name = 'xxx'

wandb.init(project=project_name, name="xxx"+str(it))
# wandb.init(project=project_name, id='uxo7savy', resume="must")


import numpy as np
import pandas as pd 
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pickle
import lzma
import random
import gzip
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.current_device()

def load_data():
    random.seed(929)
    
    data_path_a = '/llm_datasets/news_full_0to80.pkl' 
    data_path_b = '/llm_datasets/amz_0to80.pkl' 
    data_path_c1 = '/llm_datasets/arxivfull1k_0to80.pkl' 
    data_path_c2 = '/llm_datasets/arxivabs1k_0to80.pkl'  

    with open(data_path_a, 'rb') as file:
        data_a = pickle.load(file)
    data_a=data_a.tolist()

    with open(data_path_b, 'rb') as file:
        data_b = pickle.load(file)
    # data_b=data_b.tolist()

    with open(data_path_c1, 'rb') as file:
        data_c1 = pickle.load(file)
    data_c1=data_c1.tolist()

    with open(data_path_c2, 'rb') as file:
        data_c2 = pickle.load(file)
    data_c2=data_c2.tolist()

    # 	data_b = pd.read_pickle(data_path_b)
    # 	data_c1 = pd.read_pickle(data_path_c1).tolist()
    # 	data_c2 = pd.read_pickle(data_path_c2).tolist()
    data_c = data_c1 + data_c2

    data_path_d = '/llm_datasets/real_books_2M.pkl' 
    data_path_e = '/llm_datasets/wiki_0to80.pkl' 
    data_path_f1 = '/llm_datasets/pubmedfull_0to80.pkl' 
    data_path_f2 = '/llm_datasets/pubmedabs_0to80.pkl'  

    with open(data_path_d, 'rb') as file:
        data_d = pickle.load(file)
    data_d=data_d.tolist()

    with open(data_path_e, 'rb') as file:
        data_e = pickle.load(file)
    data_e=data_e.tolist()

    with open(data_path_f1, 'rb') as file:
        data_f1 = pickle.load(file)
    data_f1=data_f1.tolist()

    with open(data_path_f2, 'rb') as file:
        data_f2 = pickle.load(file)
    data_f2=data_f2.tolist()

    # 	data_b = pd.read_pickle(data_path_b)
    # 	data_c1 = pd.read_pickle(data_path_c1).tolist()
    # 	data_c2 = pd.read_pickle(data_path_c2).tolist()
    data_f = data_f1 + data_f2
    
    data_path_g = '/llm_datasets/github_py24.pkl' 
    data_path_o1 = '/llm_datasets/ready1k_owtc_3M.pkl'  
    data_path_o5 = '/llm_datasets/ready500_owtc.pkl' 

    with open(data_path_g, 'rb') as file:
        data_g = pickle.load(file)

    with open(data_path_o1, 'rb') as file:
        data_o1 = pickle.load(file)
    data_o1=data_o1.tolist()

    with open(data_path_o5, 'rb') as file:
        data_o5 = pickle.load(file)
    data_o5=data_o5.tolist()

    random.shuffle(data_a)
    random.shuffle(data_b)
    random.shuffle(data_c)
    random.shuffle(data_d)
    random.shuffle(data_e)
    random.shuffle(data_f)
    random.shuffle(data_g)
    random.shuffle(data_o1)
    random.shuffle(data_o5)

    number_of_samples = int(400600) # sample 1m from each domain
    number_of_samples2 = int(180308) # sample 1m from each domain
    data_a = random.sample(data_a, number_of_samples)
    data_b = random.sample(data_b, number_of_samples)
    data_c = random.sample(data_c, number_of_samples)

    data_d = random.sample(data_d, number_of_samples)
    data_e = random.sample(data_e, number_of_samples)
    data_f = random.sample(data_f, number_of_samples)
    
    data_g = random.sample(data_g, number_of_samples)
    data_o1 = random.sample(data_o1, number_of_samples)
    data_o5 = random.sample(data_o5, number_of_samples)

    return data_a, data_b, data_c, data_d, data_e, data_f, data_g, data_o1, data_o5



if __name__ == '__main__':
    print("start")

    data_a, data_b, data_c, data_d, data_e, data_f, data_g, data_o1, data_o5 = load_data()

    # split data into train and val
    test_ratio = 0.002
    test_ratio2 = 0.0015

    train_data_a, val_data_a = train_test_split(data_a, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_b, val_data_b = train_test_split(data_b, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_c, val_data_c = train_test_split(data_c, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_d, val_data_d = train_test_split(data_d, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_e, val_data_e = train_test_split(data_e, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_f, val_data_f = train_test_split(data_f, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_g, val_data_g = train_test_split(data_g, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_o1, val_data_o1 = train_test_split(data_o1, test_size=test_ratio, shuffle=True, random_state=40)
    train_data_o5, val_data_o5 = train_test_split(data_o5, test_size=test_ratio, shuffle=True, random_state=40)

    # data_mlm = train_data_a + train_data_b + train_data_c1
    data_mlm = train_data_a[:344000] + train_data_b[:285000] + train_data_c[:193000] + train_data_d[:121000] + train_data_e[:30000] + train_data_f[:245000] + train_data_g[:146000] + train_data_o1[:380000] + train_data_o5[:56000]
    random.shuffle(data_mlm)
    # model_name = "bert-base-uncased"
    model_name = "bert-large-uncased"
    
    tokenizer_mlm = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

    print("Tokenize training data....")
    train_mlm = tokenizer_mlm(data_mlm, return_tensors="pt", padding=True, truncation=True, max_length=295)
    trainset = Dataset.from_dict(train_mlm)
    # 	with open('tokens_train_mlm_nx200k.pkl', 'wb') as file:
    # 		pickle.dump(train_mlm, file, protocol=4)

    print("Tokenize validation data....")
    eval_mlm_a = tokenizer_mlm(val_data_a, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_b = tokenizer_mlm(val_data_b, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_c = tokenizer_mlm(val_data_c, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_d = tokenizer_mlm(val_data_d, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_e = tokenizer_mlm(val_data_e, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_f = tokenizer_mlm(val_data_f, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_g = tokenizer_mlm(val_data_g, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_o1 = tokenizer_mlm(val_data_o1, return_tensors="pt", padding=True, truncation=True, max_length=295)
    eval_mlm_o5 = tokenizer_mlm(val_data_o5, return_tensors="pt", padding=True, truncation=True, max_length=295)

    val_a = Dataset.from_dict(eval_mlm_a)
    val_b = Dataset.from_dict(eval_mlm_b)
    val_c = Dataset.from_dict(eval_mlm_c)
    val_d = Dataset.from_dict(eval_mlm_d)
    val_e = Dataset.from_dict(eval_mlm_e)
    val_f = Dataset.from_dict(eval_mlm_f)
    val_g = Dataset.from_dict(eval_mlm_g)
    val_o1 = Dataset.from_dict(eval_mlm_o1)
    val_o5 = Dataset.from_dict(eval_mlm_o5)
    eval_dataset_dict = {'news': val_a, 'amazon': val_b, 'arxiv': val_c, 'books': val_d, 'wiki': val_e, 'pubmed': val_f, 'github': val_g, 'owtc1k': val_o1, 'owtc500': val_o5}
    print(trainset)
    print(eval_dataset_dict)
    print("data is ready.")


    #####################  Training Arguments ############

    batch_size = 48
    epochs_mlm = 1
    # learning_rate_mlm = 1e-4
    # learning_rate_mlm = 3e-5
    learning_rate_mlm = 1e-4


    data_collator_mlm = DataCollatorForLanguageModeling(
        tokenizer=tokenizer_mlm,
        mlm=True,
        mlm_probability=0.15,  # Mask 15% of tokens,
        return_tensors="pt"
    )

    # Create a training configuration for MLM fine-tuning
    output_dir = "./xxx-iter-"+str(it)+"_log"
    save_dir = "./xxx-iter-"+str(it)

    training_args_mlm = TrainingArguments(
        output_dir=output_dir,
        # overwrite_output_dir=True,
        # save_total_limit=2,
        num_train_epochs=epochs_mlm,
        per_device_train_batch_size=batch_size,
        save_strategy='steps',
        save_steps=1000,
        evaluation_strategy='steps',
        eval_steps=500,
        logging_strategy='steps',
        logging_steps=500,
        learning_rate=learning_rate_mlm,
        lr_scheduler_type='linear',
        optim='adamw_torch',
        report_to='wandb',
        logging_first_step = True
        # resume_from_checkpoint=output_dir+'/checkpoint-10000',
    )

    # Load the pre-trained BERT model for MLM fine-tuning
    model_mlm = AutoModelForMaskedLM.from_pretrained(model_name).to(device)
    # model_mlm = AutoModelForMaskedLM.from_pretrained(output_dir+'/checkpoint-10000').to(device)

    print(output_dir)
    print(save_dir)
    print(epochs_mlm)


    ################### Training ################

    trainer_mlm = Trainer(
        model=model_mlm,
        args=training_args_mlm,
        data_collator=data_collator_mlm,
        train_dataset=trainset,
        eval_dataset=eval_dataset_dict

    )

    
    trainer_mlm.train()
    trainer_mlm.save_model(save_dir)
    print("model is saved to: ", save_dir)
    print("logs are saved to:", output_dir)

### Evaluation 

In [ ]:
def load_data_val():
    random.seed(920)
    
    data_path_a = '/llm_datasets/news_full_0to80.pkl' 
    data_path_b = '/llm_datasets/amz_0to80.pkl' 
    data_path_c1 = '/llm_datasets/arxivfull1k_0to80.pkl' 
    data_path_c2 = '/llm_datasets/arxivabs1k_0to80.pkl'  

    with open(data_path_a, 'rb') as file:
        data_a = pickle.load(file)
    data_a=data_a.tolist()

    with open(data_path_b, 'rb') as file:
        data_b = pickle.load(file)
    # data_b=data_b.tolist()

    with open(data_path_c1, 'rb') as file:
        data_c1 = pickle.load(file)
    data_c1=data_c1.tolist()

    with open(data_path_c2, 'rb') as file:
        data_c2 = pickle.load(file)
    data_c2=data_c2.tolist()

    # 	data_b = pd.read_pickle(data_path_b)
    # 	data_c1 = pd.read_pickle(data_path_c1).tolist()
    # 	data_c2 = pd.read_pickle(data_path_c2).tolist()
    data_c = data_c1 + data_c2

    data_path_d = '/llm_datasets/real_books_2M.pkl' 
    data_path_e = '/llm_datasets/wiki_0to80.pkl' 
    data_path_f1 = '/llm_datasets/pubmedfull_0to80.pkl' 
    data_path_f2 = '/llm_datasets/pubmedabs_0to80.pkl'  

    with open(data_path_d, 'rb') as file:
        data_d = pickle.load(file)
    data_d=data_d.tolist()

    with open(data_path_e, 'rb') as file:
        data_e = pickle.load(file)
    data_e=data_e.tolist()

    with open(data_path_f1, 'rb') as file:
        data_f1 = pickle.load(file)
    data_f1=data_f1.tolist()

    with open(data_path_f2, 'rb') as file:
        data_f2 = pickle.load(file)
    data_f2=data_f2.tolist()

    # 	data_b = pd.read_pickle(data_path_b)
    # 	data_c1 = pd.read_pickle(data_path_c1).tolist()
    # 	data_c2 = pd.read_pickle(data_path_c2).tolist()
    data_f = data_f1 + data_f2
    
    data_path_g = '/llm_datasets/github_py24.pkl' 
    data_path_o1 = '/llm_datasets/ready1k_owtc_3M.pkl'  
    data_path_o5 = '/llm_datasets/ready500_owtc.pkl' 

    with open(data_path_g, 'rb') as file:
        data_g = pickle.load(file)

    with open(data_path_o1, 'rb') as file:
        data_o1 = pickle.load(file)
    data_o1=data_o1.tolist()

    with open(data_path_o5, 'rb') as file:
        data_o5 = pickle.load(file)
    data_o5=data_o5.tolist()

    random.shuffle(data_a)
    random.shuffle(data_b)
    random.shuffle(data_c)
    random.shuffle(data_d)
    random.shuffle(data_e)
    random.shuffle(data_f)
    random.shuffle(data_g)
    random.shuffle(data_o1)
    random.shuffle(data_o5)

    number_of_samples = int(60600) # sample 1m from each domain
    number_of_samples2 = int(180308) # sample 1m from each domain
    data_a = random.sample(data_a, number_of_samples)
    data_b = random.sample(data_b, number_of_samples)
    data_c = random.sample(data_c, number_of_samples)

    data_d = random.sample(data_d, number_of_samples)
    data_e = random.sample(data_e, number_of_samples)
    data_f = random.sample(data_f, number_of_samples)
    
    data_g = random.sample(data_g, number_of_samples)
    data_o1 = random.sample(data_o1, number_of_samples)
    data_o5 = random.sample(data_o5, number_of_samples)

    return data_a, data_b, data_c, data_d, data_e, data_f, data_g, data_o1, data_o5


data_a, data_b, data_c, data_d, data_e, data_f, data_g, data_o1, data_o5 = load_data_val()

# split data into train and val
test_ratio = 0.99
test_ratio2 = 0.0015

train_data_a, val_data_a = train_test_split(data_a, test_size=test_ratio, shuffle=True, random_state=40)
train_data_b, val_data_b = train_test_split(data_b, test_size=test_ratio, shuffle=True, random_state=40)
train_data_c, val_data_c = train_test_split(data_c, test_size=test_ratio, shuffle=True, random_state=40)
train_data_d, val_data_d = train_test_split(data_d, test_size=test_ratio, shuffle=True, random_state=40)
train_data_e, val_data_e = train_test_split(data_e, test_size=test_ratio, shuffle=True, random_state=40)
train_data_f, val_data_f = train_test_split(data_f, test_size=test_ratio, shuffle=True, random_state=40)
train_data_g, val_data_g = train_test_split(data_g, test_size=test_ratio, shuffle=True, random_state=40)
train_data_o1, val_data_o1 = train_test_split(data_o1, test_size=test_ratio, shuffle=True, random_state=40)
train_data_o5, val_data_o5 = train_test_split(data_o5, test_size=test_ratio, shuffle=True, random_state=40)


model_name = "bert-large-uncased"
tokenizer_mlm = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)


print("Tokenize validation data....")
eval_mlm_a = tokenizer_mlm(val_data_a[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_b = tokenizer_mlm(val_data_b[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_c = tokenizer_mlm(val_data_c[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_d = tokenizer_mlm(val_data_d[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_e = tokenizer_mlm(val_data_e[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_f = tokenizer_mlm(val_data_f[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_g = tokenizer_mlm(val_data_g[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_o1 = tokenizer_mlm(val_data_o1[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)
eval_mlm_o5 = tokenizer_mlm(val_data_o5[:50000], return_tensors="pt", padding=True, truncation=True, max_length=295)

val_a = Dataset.from_dict(eval_mlm_a)
val_b = Dataset.from_dict(eval_mlm_b)
val_c = Dataset.from_dict(eval_mlm_c)
val_d = Dataset.from_dict(eval_mlm_d)
val_e = Dataset.from_dict(eval_mlm_e)
val_f = Dataset.from_dict(eval_mlm_f)
val_g = Dataset.from_dict(eval_mlm_g)
val_o1 = Dataset.from_dict(eval_mlm_o1)
val_o5 = Dataset.from_dict(eval_mlm_o5)
eval_dataset_dict = {'news': val_a, 'amazon': val_b, 'arxiv': val_c, 'books': val_d, 'wiki': val_e, 'pubmed': val_f, 'github': val_g, 'owtc1k': val_o1, 'owtc500': val_o5}
print(trainset)
print(eval_dataset_dict)
print("data is ready.")

# trainer_mlm.eval_dataset = eval_dataset_dict

In [ ]:
eval_dataset_dict_s = eval_dataset_dict


rest2 = np.zeros([10,1])
i = 0
for name, dataset in eval_dataset_dict_s.items():
    print(f"Running evaluation on {name}")
    trainer_mlm.eval_dataset = dataset
    result = trainer_mlm.evaluate()
    rest2[i,0] = np.exp(result['eval_loss'])
    i += 1
    print(f"Results for {name}: {result}")

rest2[9,0] = np.average(rest2[:9,0])
rest2